# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secretofthedev import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [4]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/advanced_stats/{symbol}?token={IEX_CLOUD_API_TOKEN}'
print(api_url)

# api documentation: https://iexcloud.io/docs/core/ADVANCED_STATS

data = requests.get(api_url).json()
data

https://api.iex.cloud/v1/data/core/advanced_stats/AAPL?token=pk_b868552dd32549b299eb1e749ea12fe2


[{'beta': 1.0625271952565498,
  'totalCash': 73100000000,
  'currentDebt': 133973000000,
  'revenue': 385706000000,
  'grossProfit': 173671000000,
  'totalRevenue': 385706000000,
  'EBITDA': 130109000000,
  'revenuePerShare': 24.98,
  'revenuePerEmployee': 2623850.34,
  'debtToEquity': 4.770769230769231,
  'profitMargin': 0.2616319165374664,
  'enterpriseValue': 2653574179980,
  'enterpriseValueToRevenue': 6.88,
  'priceToSales': 6.79,
  'priceToBook': 35.34,
  'forwardPERatio': 25.714286469565884,
  'pegRatio': 2.825267418477828,
  'peHigh': 16.680636344332044,
  'peLow': 10.616518420946807,
  'week52highDate': '2023-12-14',
  'week52lowDate': '2023-04-12',
  'putCallRatio': None,
  'companyName': 'Apple Inc',
  'marketcap': 2618634179980,
  'week52high': 199.11,
  'week52low': 158.31,
  'week52highSplitAdjustOnly': 199.62,
  'week52highDateSplitAdjustOnly': '2023-12-14',
  'week52lowSplitAdjustOnly': 159.78,
  'week52lowDateSplitAdjustOnly': '2023-04-12',
  'week52change': 0.03946510

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [5]:
data[0]['year1ChangePercent']

0.03946510270832215

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [ ]:
# # Function sourced from 
# # https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
# def chunks(lst, n):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), n):
#         yield lst[i:i + n]   
        
# symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_strings = []
# for i in range(0, len(symbol_groups)):
#     symbol_strings.append(','.join(symbol_groups[i]))
# #     print(symbol_strings[i])

# my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']



Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [35]:
# Chunks not needed for new updated IEX Cloud API, we can simply use batch API as per below

my_columns = ['Ticker', 'Company Name', 'Company Name V', 'Stock Price', 'One-year price return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)


symbol_list = stocks['Ticker'].to_list()
symbol_string = ','.join(symbol_list)
batch_api_url = f'https://api.iex.cloud/v1/data/core/ADVANCED_STATS,QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
print(batch_api_url)
final_dataframe

https://api.iex.cloud/v1/data/core/ADVANCED_STATS,QUOTE/A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,H

,Ticker,Company Name,Company Name V,Stock Price,One-year price return,Number of Shares to Buy


Fetching data and data validation

In [36]:
data = requests.get(batch_api_url).json()
print(len(data))
print(len(symbol_list))
nr_of_endpoints = 2
if len(data) == nr_of_endpoints * len(symbol_list):
    split = len(data) // nr_of_endpoints
    data_advanced_stats = data[:split]
    data_quote = data[split:]
    for stock_stats, stock_quote in zip(data_advanced_stats, data_quote):
        final_dataframe = pd.concat([final_dataframe, pd.DataFrame([[stock_quote['symbol'], stock_quote['companyName'], stock_stats['companyName'], stock_quote['latestPrice'], stock_stats['year1ChangePercent'], 'N/A']], columns = my_columns)], ignore_index = True)

final_dataframe

1010
505


/var/folders/_5/hcq3myc97jn9bf7mbp59_jn00000gn/T/ipykernel_32509/1421066058.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_dataframe = pd.concat([final_dataframe, pd.DataFrame([[stock_quote['symbol'], stock_quote['companyName'], stock_stats['companyName'], stock_quote['latestPrice'], stock_stats['year1ChangePercent'], 'N/A']], columns = my_columns)], ignore_index = True)


,Ticker,Company Name,Company Name V,Stock Price,One-year price return,Number of Shares to Buy
0,A,Agilent Technologies Inc.,Agilent Technologies Inc.,144.12,0.051703,N/A
1,AAL,American Airlines Group Inc,American Airlines Group Inc,13.76,-0.017844,N/A
2,AAP,Advance Auto Parts Inc,Advance Auto Parts Inc,77.34,-0.342978,N/A
3,AAPL,Apple Inc,Apple Inc,169.58,0.039465,N/A
4,ABBV,Abbvie Inc,Abbvie Inc,170.00,0.094869,N/A
...,...,...,...,...,...,...
500,YUM,Yum Brands Inc.,Yum Brands Inc.,136.10,0.050085,N/A
501,ZBH,Zimmer Biomet Holdings Inc,Zimmer Biomet Holdings Inc,127.33,-0.006624,N/A
502,ZBRA,Zebra Technologies Corp. - Class A,Zebra Technologies Corp.,290.38,-0.032905,N/A
503,ZION,Zions Bancorporation N.A,Zions Bancorporation N.A,42.08,0.507482,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [37]:
final_dataframe.sort_values('One-year price return', ascending=False, inplace=True) # inplace=True to modify the original dataframe
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop=True, inplace=True)
final_dataframe

,Ticker,Company Name,Company Name V,Stock Price,One-year price return,Number of Shares to Buy
0,LB,L Brands Inc,L Brands Inc,79.92,2.281287,N/A
1,NVDA,NVIDIA Corp,NVIDIA Corp,880.08,2.256292,N/A
2,GPS,"Gap, Inc.","Gap, Inc.",24.25,1.694474,N/A
3,SLG,SL Green Realty Corp.,SL Green Realty Corp.,51.73,1.412454,N/A
4,AVGO,Broadcom Inc,Broadcom Inc,1339.43,1.225013,N/A
5,RCL,Royal Caribbean Group,Royal Caribbean Group,136.02,1.219648,N/A
6,NRG,NRG Energy Inc.,NRG Energy Inc.,72.77,1.170289,N/A
7,ETN,Eaton Corporation plc,Eaton Corporation plc,330.51,1.149812,N/A
8,LLY,Lilly(Eli) & Co,Lilly(Eli) & Co,784.21,1.147152,N/A
9,MU,Micron Technology Inc.,Micron Technology Inc.,123.58,1.133091,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [39]:
def portfolio_input():
    global portfolio_size
    while True:
        try:
            portfolio_size = input('Enter the value of your portfolio:')
            val = float(portfolio_size)
            print(f'Portfolio value saved: {val}')
            break
        except ValueError:
            print('That is not a number. \nPlease try again.')
            
portfolio_input()
print(portfolio_size)

Portfolio value saved: 1000000.0
1000000


In [40]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Company Name,Company Name V,Stock Price,One-year price return,Number of Shares to Buy
0,LB,L Brands Inc,L Brands Inc,79.92,2.281287,245
1,NVDA,NVIDIA Corp,NVIDIA Corp,880.08,2.256292,22
2,GPS,"Gap, Inc.","Gap, Inc.",24.25,1.694474,808
3,SLG,SL Green Realty Corp.,SL Green Realty Corp.,51.73,1.412454,379
4,AVGO,Broadcom Inc,Broadcom Inc,1339.43,1.225013,14
5,RCL,Royal Caribbean Group,Royal Caribbean Group,136.02,1.219648,144
6,NRG,NRG Energy Inc.,NRG Energy Inc.,72.77,1.170289,269
7,ETN,Eaton Corporation plc,Eaton Corporation plc,330.51,1.149812,59
8,LLY,Lilly(Eli) & Co,Lilly(Eli) & Co,784.21,1.147152,25
9,MU,Micron Technology Inc.,Micron Technology Inc.,123.58,1.133091,158


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [89]:
hqm_columns = [
                'Ticker', 
                'Company Name',
                'Company Name V',
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

symbol_list = stocks['Ticker'].to_list()
symbol_string = ','.join(symbol_list)
hqm_batch_api_url = f'https://api.iex.cloud/v1/data/core/ADVANCED_STATS,QUOTE/{symbol_string}?token={IEX_CLOUD_API_TOKEN}'
print(batch_api_url)

hqm_data = requests.get(hqm_batch_api_url).json()
print(len(data))
print(len(symbol_list))
nr_of_endpoints = 2
if len(hqm_data) == nr_of_endpoints * len(symbol_list):
    split = len(hqm_data) // nr_of_endpoints
    hqm_data_advanced_stats = hqm_data[:split]
    hqm_data_quote = hqm_data[split:]
    for stock_stats, stock_quote in zip(hqm_data_advanced_stats, hqm_data_quote):
        hqm_dataframe = pd.concat([hqm_dataframe, pd.DataFrame([[
            stock_quote['symbol'],
            stock_quote['companyName'],
            stock_stats['companyName'],
            stock_quote['latestPrice'],
            'N/A',
            stock_stats['year1ChangePercent'],
            'N/A',
            stock_stats['month6ChangePercent'],
            'N/A',
            stock_stats['month3ChangePercent'],
            'N/A',
            stock_stats['month1ChangePercent'],
            'N/A',
            'N?A'
            ]], columns = hqm_columns)], ignore_index = True)
      
# drop N/A values  
hqm_dataframe.dropna(inplace=True)
hqm_dataframe

https://api.iex.cloud/v1/data/core/ADVANCED_STATS,QUOTE/A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA,CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,H

/var/folders/_5/hcq3myc97jn9bf7mbp59_jn00000gn/T/ipykernel_32509/233637161.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  hqm_dataframe = pd.concat([hqm_dataframe, pd.DataFrame([[


,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,Agilent Technologies Inc.,144.12,N/A,0.051703,N/A,0.306926,N/A,0.105654,N/A,-0.033192,N/A,N?A
1,AAL,American Airlines Group Inc,American Airlines Group Inc,13.76,N/A,-0.017844,N/A,0.078370,N/A,0.011765,N/A,-0.076510,N/A,N?A
2,AAP,Advance Auto Parts Inc,Advance Auto Parts Inc,77.34,N/A,-0.342978,N/A,0.501039,N/A,0.262327,N/A,0.049674,N/A,N?A
3,AAPL,Apple Inc,Apple Inc,169.58,N/A,0.039465,N/A,-0.039601,N/A,-0.061635,N/A,0.003432,N/A,N?A
4,ABBV,Abbvie Inc,Abbvie Inc,170.00,N/A,0.094869,N/A,0.169334,N/A,0.058494,N/A,-0.058537,N/A,N?A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,Yum Brands Inc.,Yum Brands Inc.,136.10,N/A,0.050085,N/A,0.150411,N/A,0.065772,N/A,-0.030489,N/A,N?A
501,ZBH,Zimmer Biomet Holdings Inc,Zimmer Biomet Holdings Inc,127.33,N/A,-0.006624,N/A,0.152501,N/A,0.063256,N/A,0.001644,N/A,N?A
502,ZBRA,Zebra Technologies Corp. - Class A,Zebra Technologies Corp.,290.38,N/A,-0.032905,N/A,0.297208,N/A,0.149155,N/A,0.006203,N/A,N?A
503,ZION,Zions Bancorporation N.A,Zions Bancorporation N.A,42.08,N/A,0.507482,N/A,0.277552,N/A,-0.034661,N/A,0.013487,N/A,N?A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [90]:
from scipy.stats import percentileofscore as score

time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])
        
# stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])

# stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])

# for i in range(len(hqm_dataframe.index)):
#     for time_period in time_periods:
#         hqm_dataframe.loc[i, f'{time_period} Return Percentile'] = 0
    
     
hqm_dataframe

,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,Agilent Technologies Inc.,144.12,N/A,0.051703,36.419753,0.306926,72.633745,0.105654,62.55144,-0.033192,20.781893,N?A
1,AAL,American Airlines Group Inc,American Airlines Group Inc,13.76,N/A,-0.017844,25.720165,0.078370,27.366255,0.011765,33.744856,-0.076510,8.024691,N?A
2,AAP,Advance Auto Parts Inc,Advance Auto Parts Inc,77.34,N/A,-0.342978,2.674897,0.501039,92.592593,0.262327,93.004115,0.049674,76.54321,N?A
3,AAPL,Apple Inc,Apple Inc,169.58,N/A,0.039465,34.567901,-0.039601,9.259259,-0.061635,13.786008,0.003432,47.942387,N?A
4,ABBV,Abbvie Inc,Abbvie Inc,170.00,N/A,0.094869,40.740741,0.169334,48.148148,0.058494,48.559671,-0.058537,11.316872,N?A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,Yum Brands Inc.,Yum Brands Inc.,136.10,N/A,0.050085,35.802469,0.150411,43.209877,0.065772,51.028807,-0.030489,23.045267,N?A
501,ZBH,Zimmer Biomet Holdings Inc,Zimmer Biomet Holdings Inc,127.33,N/A,-0.006624,27.160494,0.152501,43.621399,0.063256,50.0,0.001644,46.090535,N?A
502,ZBRA,Zebra Technologies Corp. - Class A,Zebra Technologies Corp.,290.38,N/A,-0.032905,23.868313,0.297208,71.8107,0.149155,74.691358,0.006203,50.823045,N?A
503,ZION,Zions Bancorporation N.A,Zions Bancorporation N.A,42.08,N/A,0.507482,83.950617,0.277552,69.341564,-0.034661,19.547325,0.013487,57.81893,N?A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [91]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
    
hqm_dataframe

,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,Agilent Technologies Inc.,Agilent Technologies Inc.,144.12,N/A,0.051703,36.419753,0.306926,72.633745,0.105654,62.55144,-0.033192,20.781893,48.096708
1,AAL,American Airlines Group Inc,American Airlines Group Inc,13.76,N/A,-0.017844,25.720165,0.078370,27.366255,0.011765,33.744856,-0.076510,8.024691,23.713992
2,AAP,Advance Auto Parts Inc,Advance Auto Parts Inc,77.34,N/A,-0.342978,2.674897,0.501039,92.592593,0.262327,93.004115,0.049674,76.54321,66.203704
3,AAPL,Apple Inc,Apple Inc,169.58,N/A,0.039465,34.567901,-0.039601,9.259259,-0.061635,13.786008,0.003432,47.942387,26.388889
4,ABBV,Abbvie Inc,Abbvie Inc,170.00,N/A,0.094869,40.740741,0.169334,48.148148,0.058494,48.559671,-0.058537,11.316872,37.191358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,Yum Brands Inc.,Yum Brands Inc.,136.10,N/A,0.050085,35.802469,0.150411,43.209877,0.065772,51.028807,-0.030489,23.045267,38.271605
501,ZBH,Zimmer Biomet Holdings Inc,Zimmer Biomet Holdings Inc,127.33,N/A,-0.006624,27.160494,0.152501,43.621399,0.063256,50.0,0.001644,46.090535,41.718107
502,ZBRA,Zebra Technologies Corp. - Class A,Zebra Technologies Corp.,290.38,N/A,-0.032905,23.868313,0.297208,71.8107,0.149155,74.691358,0.006203,50.823045,55.298354
503,ZION,Zions Bancorporation N.A,Zions Bancorporation N.A,42.08,N/A,0.507482,83.950617,0.277552,69.341564,-0.034661,19.547325,0.013487,57.81893,57.664609


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [92]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(inplace=True, drop=True)


In [93]:
hqm_dataframe

,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MU,Micron Technology Inc.,Micron Technology Inc.,123.58,N/A,1.133091,98.148148,0.772882,99.176955,0.482317,99.588477,0.249741,99.588477,99.125514
1,NRG,NRG Energy Inc.,NRG Energy Inc.,72.77,N/A,1.170289,98.765432,0.898488,99.588477,0.417419,98.765432,0.200429,98.148148,98.816872
2,GE,GE Aerospace,GE Aerospace,156.30,N/A,1.101857,97.942387,0.762332,98.971193,0.554548,99.794239,0.179122,97.736626,98.611111
3,WDC,Western Digital Corp.,Western Digital Corp.,73.11,N/A,1.079943,97.736626,0.575647,96.296296,0.460739,99.382716,0.137368,95.061728,97.119342
4,MPC,Marathon Petroleum Corp,Marathon Petroleum Corp,219.13,N/A,0.741168,93.82716,0.546382,95.26749,0.443925,99.176955,0.224395,98.765432,96.759259
5,ETN,Eaton Corporation plc,Eaton Corporation plc,330.51,N/A,1.149812,98.559671,0.583659,96.707819,0.411670,98.559671,0.101003,90.740741,96.141975
6,PSX,Phillips 66,Phillips 66,170.75,N/A,0.722260,93.209877,0.589539,97.119342,0.271693,94.444444,0.144360,96.296296,95.26749
7,FTI,TechnipFMC plc,TechnipFMC plc,26.93,N/A,1.019710,97.325103,0.413300,84.567901,0.360816,98.148148,0.213353,98.559671,94.650206
8,GPS,"Gap, Inc.","Gap, Inc.",24.25,N/A,1.694474,99.588477,1.382941,100.0,0.166426,77.777778,0.254527,99.794239,94.290123
9,CMG,Chipotle Mexican Grill,Chipotle Mexican Grill,2905.42,N/A,0.724070,93.415638,0.594396,97.530864,0.313006,96.502058,0.079993,86.831276,93.569959


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [94]:
portfolio_input()

Portfolio value saved: 1000000.0


In [95]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for row in hqm_dataframe.index:
    hqm_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe.loc[row, 'Price'])
    
hqm_dataframe

,Ticker,Company Name,Company Name V,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MU,Micron Technology Inc.,Micron Technology Inc.,123.58,158,1.133091,98.148148,0.772882,99.176955,0.482317,99.588477,0.249741,99.588477,99.125514
1,NRG,NRG Energy Inc.,NRG Energy Inc.,72.77,269,1.170289,98.765432,0.898488,99.588477,0.417419,98.765432,0.200429,98.148148,98.816872
2,GE,GE Aerospace,GE Aerospace,156.30,125,1.101857,97.942387,0.762332,98.971193,0.554548,99.794239,0.179122,97.736626,98.611111
3,WDC,Western Digital Corp.,Western Digital Corp.,73.11,268,1.079943,97.736626,0.575647,96.296296,0.460739,99.382716,0.137368,95.061728,97.119342
4,MPC,Marathon Petroleum Corp,Marathon Petroleum Corp,219.13,89,0.741168,93.82716,0.546382,95.26749,0.443925,99.176955,0.224395,98.765432,96.759259
5,ETN,Eaton Corporation plc,Eaton Corporation plc,330.51,59,1.149812,98.559671,0.583659,96.707819,0.411670,98.559671,0.101003,90.740741,96.141975
6,PSX,Phillips 66,Phillips 66,170.75,114,0.722260,93.209877,0.589539,97.119342,0.271693,94.444444,0.144360,96.296296,95.26749
7,FTI,TechnipFMC plc,TechnipFMC plc,26.93,728,1.019710,97.325103,0.413300,84.567901,0.360816,98.148148,0.213353,98.559671,94.650206
8,GPS,"Gap, Inc.","Gap, Inc.",24.25,808,1.694474,99.588477,1.382941,100.0,0.166426,77.777778,0.254527,99.794239,94.290123
9,CMG,Chipotle Mexican Grill,Chipotle Mexican Grill,2905.42,6,0.724070,93.415638,0.594396,97.530864,0.313006,96.502058,0.079993,86.831276,93.569959


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [96]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [97]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [98]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Company Name', string_template],
                    'D': ['Company Name V', string_template],
                    'E': ['Number of Shares to Buy', integer_template],
                    'F': ['One-Year Price Return', percent_template],
                    'G': ['One-Year Return Percentile', percent_template],
                    'H': ['Six-Month Price Return', percent_template],
                    'I': ['Six-Month Return Percentile', percent_template],
                    'J': ['Three-Month Price Return', percent_template],
                    'K': ['Three-Month Return Percentile', percent_template],
                    'L': ['One-Month Price Return', percent_template],
                    'M': ['One-Month Return Percentile', percent_template],
                    'N': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [99]:
writer.close()